In [ ]:
# Standard library imports
import os

# Set the working directory
os.chdir('/summproject/')

# Numerical & data handling
import numpy as np
import pandas as pd

# Image processing & visualization
from PIL import Image
import matplotlib.pyplot as plt

# Hugging Face Transformers
from transformers import (
    CLIPProcessor,
    CLIPModel,
    CLIPTokenizer,
    pipeline
)


/Users/m.ding-2/anaconda3/envs/flta/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
#!pip install transformers torch pillow requests
#!pip install pip install SimpleITK        

# Process the text file 
To search the corresponding text and add the file path in the dataframe 
To process the report, the sentences less than 3 words are removed. The impression and finding part are merged together as one paragrahp.

In [124]:
patient_to_select='data_summary/mimic-cxr_lung_opacity_2000sample.csv'
df_patient=pd.read_csv(patient_to_select)
df_patient['subject_id'] = 'p'+df_patient['subject_id'].astype(str)
df_patient['study_id'] = 's'+df_patient['study_id'].astype(str)
df_patient

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices
0,p12779994,s55965863,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,p11494296,s54944489,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2,p17251996,s57007998,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,1.0,NaN,1.0,NaN,0.0,NaN,NaN
3,p14130788,s58955766,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0,NaN,0.0,NaN,-1.0,0.0,NaN
4,p13293910,s56789738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,-1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,p13973055,s55847178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0,1.0
1996,p18550032,s54642925,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.0,NaN,1.0,NaN,1.0
1997,p15311611,s52627144,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.0,NaN,0.0,0.0,1.0
1998,p11486239,s51256738,NaN,1.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,1.0,0.0,NaN,NaN,NaN


In [66]:
folder='/Users/m.ding-2/Desktop/summproject/data/text/files'
subfolders=os.listdir(folder)
subfolders

['p19', 'p17', 'p10', 'p11', 'p16', 'p18', 'p13', 'p14', 'p15', 'p12']

In [ ]:

###Add the file path of the report to the dataframe

for indx,row in df_patient.iterrows():
    subject_id = row['subject_id']
    subject_id = subject_id.split('p')[1]  # Remove 'p' prefix
    
    study_id = row['study_id']
    study_id = study_id.split('s')[1]  # Remove 's' prefix

 
    for subfolder in subfolders:
        subfolder_path = os.path.join(folder, subfolder)
        files = os.listdir(subfolder_path)
        if subject_id in files:
            file_path = os.path.join(subfolder_path, subject_id, study_id + '.txt')
            if os.path.exists(file_path):
                print(f"File found: {file_path}")
            df_patient.at[indx, 'file_path'] = file_path
            break

    


File found: /Users/m.ding-2/Desktop/summproject/data/text/files/p12/p12779994/s55965863.txt
File found: /Users/m.ding-2/Desktop/summproject/data/text/files/p11/p11494296/s54944489.txt
File found: /Users/m.ding-2/Desktop/summproject/data/text/files/p17/p17251996/s57007998.txt
File found: /Users/m.ding-2/Desktop/summproject/data/text/files/p14/p14130788/s58955766.txt
File found: /Users/m.ding-2/Desktop/summproject/data/text/files/p13/p13293910/s56789738.txt
File found: /Users/m.ding-2/Desktop/summproject/data/text/files/p11/p11690358/s56825148.txt
File found: /Users/m.ding-2/Desktop/summproject/data/text/files/p13/p13350579/s58780206.txt
File found: /Users/m.ding-2/Desktop/summproject/data/text/files/p16/p16783577/s50730772.txt
File found: /Users/m.ding-2/Desktop/summproject/data/text/files/p10/p10440642/s56122537.txt
File found: /Users/m.ding-2/Desktop/summproject/data/text/files/p14/p14306557/s58579716.txt
File found: /Users/m.ding-2/Desktop/summproject/data/text/files/p15/p15390826/s5

In [142]:
import re
from pathlib import Path

def extract_findings_and_impression(text: str) -> str:
    """
    Extract and concatenate the 'Findings' and 'Impression' sections from a radiology report.

    Sections are located by their headings (case-insensitive), and all text between each heading
    and the next ALL-CAPS heading (or end-of-text) is captured.
    """
    combined = []
    for section in ("FINDINGS", "IMPRESSION"):
        # Regex: match 'SECTION:' then lazily up to next all-caps heading ending with ':' or end of text
        pattern = rf"(?is){section}:(.*?)(?=\n[A-Z ]+?:|\Z)"
        match = re.search(pattern, text)
        if match:
            combined.append(match.group(1).strip())
    if len(combined) > 0:
      
        return " ".join(combined)
    if len(combined) == 0:
        return text.strip()


def split_and_filter_sentences(text: str, min_words: int = 3) -> list[str]:
    """
    Split text into sentences on ., !, or ? and remove any sentence with fewer than min_words words.
    """
    # Split on punctuation followed by whitespace
    sentences = re.split(r"(?<=[\.!?])\s+", text)
    # Filter out short sentences
    return [s.strip() for s in sentences if len(s.split()) >= min_words]




In [135]:
"""
Script: Encode text using CLIP text encoder to obtain a fixed-length embedding vector.

Steps:
1. Load CLIP model and processor.
2. Tokenize and preprocess the input text.
3. Extract the 512-dim L2-normalized embedding using the text encoder.
"""

import torch
from transformers import CLIPProcessor, CLIPModel


# 1) Load CLIP model & processor
MODEL_NAME = "openai/clip-vit-base-patch32"
processor = CLIPProcessor.from_pretrained(MODEL_NAME)
model = CLIPModel.from_pretrained(MODEL_NAME).eval()


def get_text_embedding(text: str) -> torch.Tensor:
    """
    Encode a single string (or sentence) into a 512-dim CLIP text embedding.

    Args:
        text (str): The input text to encode.

    Returns:
        torch.Tensor: A (1, 512) L2-normalized text embedding tensor.
    """
    # 2) Tokenize and create tensors
    inputs = processor(text=text,
                       return_tensors="pt",
                       padding=True,
                       truncation=True,
                       max_length=77)  # CLIP's max token length

    # 3) Extract text features
    with torch.no_grad():
        text_embeds = model.get_text_features(**inputs)  # (1, 512)
        text_embeds = text_embeds / text_embeds.norm(dim=-1, keepdim=True)

    return text_embeds


# Extracting CLIP Embedding Vectors

CLIP’s text encoder accepts at most **77 tokens**. To ensure our radiology reports fit this limit, we compare two strategies:

1. **Sentence-Level Embeddings + Mean Pooling**  
   - Split the full report into individual sentences.  
   - Feed each sentence separately through CLIP to obtain its 512-dim embedding.  
   - Aggregate by computing the **mean** of all sentence embeddings to form a single report embedding.

2. **Abstractive Summarization**  
   - Use a pretrained summarization model (e.g. BART from Hugging Face) to condense the entire report into a summary of **≤ 77 tokens**.  
   - Feed the resulting summary into CLIP to obtain one 512-dim embedding.  


In [143]:
def get_pool_embedding(file_path):
        # 1) Path to your report
    report_path = Path(file_path)

    # 2) Read the report text
    report_text = report_path.read_text(encoding="utf-8")

    # 3) Extract and combine Findings & Impression
    combined_text = extract_findings_and_impression(report_text)

    # 4) Split into sentences and filter
    sentences = split_and_filter_sentences(combined_text, min_words=3)


    text_embeds= get_text_embedding(sentences)
    txt_embed = text_embeds.mean(dim=0, keepdim=True)  # (1,512)
    txt_embed = txt_embed / txt_embed.norm(dim=-1, keepdim=True)
    return txt_embed


In [ ]:
##sentence level embedding
for indx, row in df_patient.iterrows():
    print(f'processing {indx+1}/{df_patient.shape[0]}: {row["subject_id"]}, {row["study_id"]}')
    file_path = row['file_path']
    txt_embed = get_pool_embedding(file_path)
    output_dir='/Users/m.ding-2/Desktop/summproject/data/text/embeddings/pool_mean'
    os.makedirs(output_dir, exist_ok=True)

    subject_id = row['subject_id']
    subject_id = subject_id.split('p')[1]  # Remove 'p' prefix
    
    study_id = row['study_id']
    study_id = study_id.split('s')[1]  # Remove 's' prefix

 
    output_path = os.path.join(output_dir, f"{subject_id}_{study_id}.npy")
    np.save(output_path,txt_embed)




processing 1/2000: p12779994, s55965863
processing 2/2000: p11494296, s54944489
processing 3/2000: p17251996, s57007998
processing 4/2000: p14130788, s58955766
processing 5/2000: p13293910, s56789738
processing 6/2000: p11690358, s56825148
processing 7/2000: p13350579, s58780206
processing 8/2000: p16783577, s50730772
processing 9/2000: p10440642, s56122537
processing 10/2000: p14306557, s58579716
processing 11/2000: p15390826, s53461741
processing 12/2000: p14783057, s50769888
processing 13/2000: p14470386, s58832223
processing 14/2000: p19299068, s59551981
processing 15/2000: p12806204, s55096288
processing 16/2000: p14512945, s55403016
processing 17/2000: p15971063, s59693154
processing 18/2000: p17250375, s50690093
processing 19/2000: p17846379, s55530921
processing 20/2000: p11682251, s55732670
processing 21/2000: p11644052, s51062850
processing 22/2000: p13999026, s52033062
processing 23/2000: p16310288, s50080690
processing 24/2000: p13180695, s51339045
processing 25/2000: p1897


**Abstractive Summarization**  



In [ ]:

def count_clip_tokens(text: str, model_name: str = "openai/clip-vit-base-patch32") -> int:
    """
    Count the number of CLIP subword tokens for a given text string.
    """
    tokenizer = CLIPTokenizer.from_pretrained(model_name)
    # Tokenize without truncation to get the full token list
    tokens = tokenizer.tokenize(text)
    return len(tokens)

def is_over_token_limit(text: str, limit: int = 77, model_name: str = "openai/clip-vit-base-patch32") -> bool:
    """
    Return True if the text exceeds the specified CLIP token limit.
    """
    num_tokens = count_clip_tokens(text, model_name)
    print(f"Token count: {num_tokens} (limit = {limit})")
    return num_tokens > limit



def extract_summary(sentences_whole):
    #use the bart to summarize the text if it exceeds the token limit (77)
    over_limit = is_over_token_limit(sentences_whole, limit=77)
    if over_limit:
        summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

        # 2) Summarize, forcing it to output ≤60 “words” (approx ⪅77 tokens)
        summary = summarizer(
            sentences_whole,
            max_length=77, 
            min_length=20, 
            do_sample=False
      
        )[0]["summary_text"]

        return summary
    else:
        return  sentences_whole



In [164]:
def get_pool_embedding_from_summary(file_path):
        # 1) Path to your report
    report_path = Path(file_path)

    # 2) Read the report text
    report_text = report_path.read_text(encoding="utf-8")

    # 3) Extract and combine Findings & Impression
    combined_text = extract_findings_and_impression(report_text)

    # 4) Split into sentences and filter
    sentences = split_and_filter_sentences(combined_text, min_words=3)
    sentences_whole = " ".join(sentences)
    sentences_whole = extract_summary(sentences_whole)

    text_embeds= get_text_embedding(sentences)
    txt_embed = text_embeds.mean(dim=0, keepdim=True)  # (1,512)
    txt_embed = txt_embed / txt_embed.norm(dim=-1, keepdim=True)
    return txt_embed


In [170]:
df_patient

,subject_id,study_id,Atelectasis,Cardiomegaly,Consolidation,Edema,Enlarged Cardiomediastinum,Fracture,Lung Lesion,Lung Opacity,No Finding,Pleural Effusion,Pleural Other,Pneumonia,Pneumothorax,Support Devices,file_path
0,p12779994,s55965863,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,/Users/m.ding-2/Desktop/summproject/data/text/...
1,p11494296,s54944489,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,/Users/m.ding-2/Desktop/summproject/data/text/...
2,p17251996,s57007998,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,1.0,NaN,1.0,NaN,0.0,NaN,NaN,/Users/m.ding-2/Desktop/summproject/data/text/...
3,p14130788,s58955766,NaN,NaN,NaN,NaN,0.0,NaN,NaN,1.0,NaN,0.0,NaN,-1.0,0.0,NaN,/Users/m.ding-2/Desktop/summproject/data/text/...
4,p13293910,s56789738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,-1.0,NaN,NaN,/Users/m.ding-2/Desktop/summproject/data/text/...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,p13973055,s55847178,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,0.0,1.0,/Users/m.ding-2/Desktop/summproject/data/text/...
1996,p18550032,s54642925,NaN,1.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.0,NaN,1.0,NaN,1.0,/Users/m.ding-2/Desktop/summproject/data/text/...
1997,p15311611,s52627144,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,1.0,NaN,0.0,0.0,1.0,/Users/m.ding-2/Desktop/summproject/data/text/...
1998,p11486239,s51256738,NaN,1.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,1.0,0.0,NaN,NaN,NaN,/Users/m.ding-2/Desktop/summproject/data/text/...


In [ ]:

for indx, row in df_patient.iterrows():
    print(f'processing {indx+1}/{df_patient.shape[0]}: {row["subject_id"]}, {row["study_id"]}')
    file_path = row['file_path']

    
    subject_id = row['subject_id']
    subject_id = subject_id.split('p')[1]  # Remove 'p' prefix
    study_id = row['study_id']
    study_id = study_id.split('s')[1]  # Remove 's' prefix
    output_dir='/Users/m.ding-2/Desktop/summproject/data/text/embeddings/bart_summary'
    output_path = os.path.join(output_dir, f"{subject_id}_{study_id}.npy")
    if not os.path.exists(output_path):
        txt_embed = get_pool_embedding_from_summary(file_path)

        os.makedirs(output_dir, exist_ok=True)

        subject_id = row['subject_id']
        subject_id = subject_id.split('p')[1]  # Remove 'p' prefix
        
        study_id = row['study_id']
        study_id = study_id.split('s')[1]  # Remove 's' prefix

    

        np.save(output_path,txt_embed)





processing 1/2000: p12779994, s55965863
processing 2/2000: p11494296, s54944489
processing 3/2000: p17251996, s57007998
processing 4/2000: p14130788, s58955766
processing 5/2000: p13293910, s56789738
processing 6/2000: p11690358, s56825148
processing 7/2000: p13350579, s58780206
processing 8/2000: p16783577, s50730772
processing 9/2000: p10440642, s56122537
processing 10/2000: p14306557, s58579716
processing 11/2000: p15390826, s53461741
processing 12/2000: p14783057, s50769888
processing 13/2000: p14470386, s58832223
processing 14/2000: p19299068, s59551981
processing 15/2000: p12806204, s55096288
processing 16/2000: p14512945, s55403016
processing 17/2000: p15971063, s59693154
processing 18/2000: p17250375, s50690093
processing 19/2000: p17846379, s55530921
processing 20/2000: p11682251, s55732670
processing 21/2000: p11644052, s51062850
processing 22/2000: p13999026, s52033062
processing 23/2000: p16310288, s50080690
processing 24/2000: p13180695, s51339045
processing 25/2000: p1897